# Overview

Let's say you wanted to generate all the two qubit Cliffords, but you don't want to use the CNOT gate, but rather the CZ gate. You have a single qubit X gate, Z gate and Phase gate as well as a Hadamard which you can apply to either or both qubits. How would you generate the full two qubit group using only these Cliffords?

For a lot of this book we will be working in the superoperator basis, so our Clifford operators are expressed as super-operators. The reason I do this is because the superoperator basis is 'projective' - it projects out the global phase. This makes it really easy to see if two operators are in fact the same. 

Juqst contains some code to generate the single qubit Cliffords directly

In [1]:
# This isn't automatically installed as part of Juqst.
# import Pkg; Pkg.add("ProgressMeter")

In [2]:
using Juqst 


using ProgressMeter
using DelimitedFiles
using LinearAlgebra

⊗ = kron # Convenience type \otimes<tab>

"""
You can check whether you have generated the gates correcly by looking at their frame.
This follows: David Gross' paper: https://arxiv.org/abs/quant-ph/0611002
If this gives 2, you have a unitary-2 design, 3 it's an orthogonal-2 design.
"""
function checkFrame(x)
    sum = 0
    @showprogress for i=1:length(x)
        for j=1:length(x)
            sum += abs(tr(x[i]'*x[j]))^4
        end
    end
    sum/(length(x)^2)
end


operatorCliffords = generateRawCliffords();
@assert round(checkFrame(operatorCliffords),digits=8) == 2
superCliffords = map(makeSuper,operatorCliffords)

# Note the makeSuper is very slow when the number of qubits become >> 4

24-element Vector{Matrix{Float64}}:
 [1.0 0.0 0.0 0.0; 0.0 0.0 0.0 -1.0; 0.0 0.0 1.0 0.0; 0.0 1.0 0.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 1.0 0.0; 0.0 -1.0 0.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 0.0 -1.0; 0.0 0.0 -1.0 0.0; 0.0 -1.0 0.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 -1.0 0.0; 0.0 1.0 0.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 -1.0 0.0 0.0; 0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 -1.0 0.0; 0.0 -1.0 0.0 0.0; 0.0 0.0 0.0 -1.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 -1.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 -1.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0]
 [1.0 0.0 0.0 0.0; 0.0 -1.0 0.0 0.0; 0.0 0.0 0.0 -1.0; 0.0 0.0 -1.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 -1.0; 0.0 0.0 1.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 -1.0 0.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 1.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 -1.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 0.0 -1.0; 0.0 1.0 0.0 0.0; 0.0 0.0 -1.0 0.0]
 [1.0 0.0 0.0 0.0; 0.0 0.0 0.0 1.0

In [3]:
# We can use stabiliser formalism to create all two qubit cliffords.

twoQubitCliffords = []
for i = 1:getNumberOfSymplecticCliffords(2)
    for j = 1:getNumberOfBitStringsCliffords(2)
        push!(twoQubitCliffords,makeFromCommand(cliffordToTableau(2,i,j)))
    end
end
print("$(length(twoQubitCliffords)) - created.\n")

# This is the slowest bit ---  you can remove this sanity check.
@assert round(checkFrame(twoQubitCliffords),digits=8) == 2

superTwoQubitCliffords = map(makeSuper,twoQubitCliffords);

11520 - created.


Progress: 100%|█████████████████████████████████████████| Time: 0:00:25


In [4]:
# Some quick and dirty functions (use a global) to help us find indexes to cliffords
# Given x, checks if its in the cliffords.
# Uses the SuperClifford to ignore phase
function findClifford(x::Array{Complex{Float64},2})
    test = makeSuper(x)
    return findfirst(x->x==test,superCliffords)
end

# Here we pass in a SuperOperator clifford
function findClifford(test::Array{Float64,2})
        return findfirst(x->x==test,superCliffords)
end


#Some basic gates

pI=[1 0;0 1]
pX=[0 1;1 0]
pZ=[1 0;0 -1]
pP=[1 0;0 im]
pH=[1 1;1 -1]/sqrt(2)
cnot12 = kron([1 0]'*[1 0],pI)+kron([0 1]'*[0 1],pX)
cnot21 = kron(pI,[1 0]'*[1 0])+kron(pX,[0 1]'*[0 1])
cZ = [1 0 0 0; 0 1 0 0;0 0 1 0;0 0 0 -1]


pXpI = kron(pX,pI)
pIpX = kron(pI,pX)
pZpI = kron(pZ,pI)
pIpZ = kron(pI,pZ)
pHpI = kron(pH,pI)
pIpH = kron(pI,pH)
pPpI = kron(pP,pI)
pIpP = kron(pI,pP)
pXX = kron(pX,pX)
pZZ = kron(pZ,pZ)
pHH = kron(pH,pH)
pPP = kron(pP,pP)




4×4 Matrix{Complex{Int64}}:
 1+0im  0+0im  0+0im   0+0im
 0+0im  0+1im  0+0im   0+0im
 0+0im  0+0im  0+1im   0+0im
 0+0im  0+0im  0+0im  -1+0im

In [5]:
# This will be filled with the minimum sequence of generators needed to generate each Clifford
minPaths=[]
for i=1:length(superTwoQubitCliffords)
    push!(minPaths,[])
end


# Lets say we want to Generate using cz
# (I think this falls into Steve's - dumbest thing possible category)
twoQubitGens = [pXpI,pIpX,pZpI,pIpZ,pHpI,pIpH,pPpI,pIpP,pZZ,pHH,pXX,cZ];
twoQubitGenString = ["XI","IX","ZI","IZ","HI","IH","PI","IP","ZZ","HH","XX","cZ"]


super2Gens = [makeSuper(x) for x in twoQubitGens]


# We have the generators, so just fill these in 
for i=1:length(super2Gens)
    # Find the index of the Clifford corresponding to the generator.
    t1=findfirst(x->x==super2Gens[i],superTwoQubitCliffords)
    minPaths[t1]=[i] # Fill it into minPaths.
end

# i.e if ZI (the third generator) corresponds to, say, Clifford 6, then the 6th entry of minPaths is set to 3.

doneOne = true

todo = count([x==[] for x in minPaths]) # number we need to find.
havedone=0

newPaths=copy(minPaths)
# We use a set to see if we have already created on, this is a lot quicker
setOfDone = Set()
# We already have the generators put them in the setOfDOne.
for i in super2Gens
    push!(setOfDone,i)
end

# We need to check if we have stalled, as if we don't give the correct generators we may not be able
# to generate them all (see RealCliffords later)
while doneOne && (havedone < todo) 
    doneOne = false
    for i in 1:length(super2Gens)
        for j in 1:length(minPaths)
              if minPaths[j]!=[] # one we have found
                  newOne = superTwoQubitCliffords[j]*super2Gens[i]
                  if !(newOne in setOfDone) # when we multiplied it by a generator did we get a previously undiscovered Clifford?
                      t1 = findfirst(x->x==newOne,superTwoQubitCliffords)
                      if newPaths[t1]==[]
                          newPaths[t1] = hcat([i],copy(minPaths[j])) # note composing direction.
                          doneOne = true
                          havedone = havedone+1
                          push!(setOfDone,newOne)
                          if havedone >= todo
                            print("Done enough\n")
                            break;
                          end
                    end
                  end
             end
        end
    end
    minPaths=copy(newPaths)
    print("That pass we now have $(count([x==[] for x in minPaths])) Cliffords left, $(length(superTwoQubitCliffords)-count([x==[] for x in minPaths])) done, max length $(maximum([length(i) for i in minPaths])).\n")
end



That pass we now have 11432 Cliffords left, 88 done, max length 2.
That pass we now have 11155 Cliffords left, 365 done, max length 3.
That pass we now have 10435 Cliffords left, 1085 done, max length 4.
That pass we now have 8821 Cliffords left, 2699 done, max length 5.
That pass we now have 5962 Cliffords left, 5558 done, max length 6.
That pass we now have 2434 Cliffords left, 9086 done, max length 7.
That pass we now have 246 Cliffords left, 11274 done, max length 8.
Done enough
That pass we now have 0 Cliffords left, 11520 done, max length 9.


In [6]:
# So for instance Clifford[356] is generated with
print(minPaths[356])


[8 12 5 7 5]

In [7]:
print("Generators for Clifford 356 are: $([twoQubitGenString[x] for x in minPaths[356]])\n")

Generators for Clifford 356 are: ["IP" "cZ" "HI" "PI" "HI"]


In [8]:
cliff = makeSuper(pI⊗pI)
for c in minPaths[356]
    cliff = super2Gens[c]*cliff
end
cliff == superTwoQubitCliffords[356]
# Again note that the the generators multiply on the left (obviously you can change line 46 of above if you want the other way round) )

true

In [9]:
# sanity check
pIpI = makeSuper(pI⊗pI)
@showprogress for x in 1:length(superTwoQubitCliffords)
    cliff = pIpI
    for c in minPaths[x]
        cliff = super2Gens[c]*cliff
    end
    @assert cliff == superTwoQubitCliffords[x]
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [10]:
# And of course it commutes through makeSuper (this is useful because makeSuper can be slow for qubits >> 4)
cliff = pI⊗pI
for c in minPaths[356]
    cliff = twoQubitGens[c]*cliff
end
@assert makeSuper(twoQubitCliffords[356]) == makeSuper(cliff)

In [ ]:
# Lets say we want to generate only the Real Cliffords (note no phase in the generators)
twoQubitRealGens = [pXpI,pIpX,pZpI,pIpZ,pHpI,pIpH,cnot12,cnot21];


#Index to the cliffords
singleReals=[]
doubleReals=[]

# The generators we will use for single qubit reals are pZ and PH
push!(singleReals,findClifford(makeSuper(pZ)))
push!(singleReals,findClifford(makeSuper(pH)))

# Note don't use the isequal predicate in findfirst as we need 0.0 to equal -0.0

let doneOne = true
    while doneOne
        doneOne=false
        toIterate = copy(singleReals)
        for i in toIterate
            n1 = findClifford(makeSuper(pZ)*superCliffords[i])
            if (findfirst(x->x==n1,singleReals) == nothing)
                push!(singleReals,n1)
                doneOne=true
            end
            n2 = findClifford(makeSuper(pH)*superCliffords[i])
            if (findfirst(x->x==n2,singleReals) == nothing)
                push!(singleReals,n2)
                doneOne=true
            end
        end
    end
end    



#Check the frame reference, this is an orthogonal 2-design, should be 3

@assert round(checkFrame([operatorCliffords[x] for x in singleReals]),digits = 8)==3


# Then generate using the generators for the real cliffords
# Note here we have both X and Z generators, they are not a minimal set
superReal2Gens = [makeSuper(x) for x in twoQubitRealGens];

doubleReals = []
for i in superReal2Gens
    push!(doubleReals,findfirst(x->x==i,superTwoQubitCliffords))
end

let doneOne = true
    while doneOne
        doneOne=false
        toIterate = copy(doubleReals)
        for i in toIterate
            for j in superReal2Gens
                lookFor = j*superTwoQubitCliffords[i]
                n1 = findfirst(x->x == lookFor,superTwoQubitCliffords)
                if (findfirst(x->x == n1,doubleReals) == nothing)
                    push!(doubleReals,n1)
                    doneOne=true
                end
            end
        end
    end
end


#Check the frame reference, this is an orthogonal 2-design, should be 3

@assert round(checkFrame([twoQubitCliffords[x] for x in doubleReals]),digits=8)==3

print("ok")


At the end of that, double reals is list of the indexes of the two qubit Cliffords that are also in the REAL Clifford group